In [ ]:
'''Objective: Compare 2 boards over time: Boot time and pass rate'''
import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import requests
from urlparse import urljoin
import kernelci_api_key

BACKEND_URL = "http://api.kernelci.org"
JOB = 'mainline'
DEFCONFIG = 'multi_v7_defconfig '
DATE_RANGE = 200
STORAGE_SERVER = 'http://storage.kernelci.org'

def invoke(board):
    headers = { "Authorization": kernelci_api_key.getkernelcikey()}
    params = {
        "job": JOB,
        "board": board,
        "defconfig":DEFCONFIG
    }
    url = urljoin(BACKEND_URL, "/boot")
    response = requests.get(url, params=params, headers=headers)
    
    #convert string resp into json, then into DataFrame
    contentjs = json.loads(response.content)
    df = json_normalize(contentjs['result'])
    
    #only keep columns of interest and rename some columns
    df2 = df[[u'_id.$oid',u'boot_result_description',u'created_on.$date',u'git_describe',u'lab_name',u'status',u'time.$date']]
    df2.columns = [u'boot_id',u'boot_result_description', u'created_on',u'git_describe', u'lab_name', u'status',u'boot_time']

    #Change created_on to DateTime. Then use this as index and sort
    df2['created_on'] = pd.to_datetime(df2['created_on'],unit='ms')
    df2.index = df2.created_on
    df2 = df2.sort_index()
    
    #drop rows for df2.status=OFFLINE
    df2 = df2[(df2.status == 'PASS') | (df2.status == 'FAIL')]
    
    #make new col of status in float
    df2['status_fl'] = df2.status.apply(lambda x: 1 if x=='PASS' else 0)

    #make new col of period. Used this to club(groupby) week
    df2['period'] = df2.created_on.apply(lambda x: pd.Period(x, 'W'))
    df2 = df2.groupby('period').mean()
    
    #Add boardname to col names so DataFrames can be merged later
    cols = [board+'_'+ x for x in df2.columns]
    df2.columns = cols
    
    return df2

In [ ]:
BOARD_1 = 'am335x-boneblack'
BOARD_2 = 'hisi-x5hd2-dkb'
df1 = invoke(BOARD_1)
df2 = invoke(BOARD_2)

In [ ]:
df3 = df1.join(df2, how='inner')
df3.head()

In [ ]:
df3[BOARD_1 + '_boot_time'].plot()
df3[BOARD_2 + '_boot_time'].plot()
plt.legend(['y='+BOARD_1,'y='+BOARD_2], loc='upper right')

In [ ]:
df3[BOARD_1 + '_status_fl'].plot()
df3[BOARD_2 + '_status_fl'].plot()
plt.legend(['y='+BOARD_1,'y='+BOARD_2], loc='lower right')
plt.ylim(0.4,1)